# Typy hietogramów

Hietogram to czasowy rozkład opadu. Hydrolog oferuje trzy typy hietogramów:

1. **Blokowy** - stała intensywność przez cały czas trwania
2. **Trójkątny** - liniowy wzrost do szczytu i spadek
3. **Beta** - rozkład Beta (najbardziej realistyczny)

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent))

from hydrolog.precipitation import BlockHietogram, TriangularHietogram, BetaHietogram

## Wspólne parametry

Wszystkie hietogramy przyjmują te same parametry generowania:
- `total_mm` - całkowita suma opadu [mm]
- `duration_min` - czas trwania opadu [min]
- `timestep_min` - krok czasowy [min]

In [ ]:
# Wspólne parametry
total_mm = 40.0       # Suma opadu
duration_min = 60.0   # Czas trwania
timestep_min = 5.0    # Krok czasowy

## 1. Hietogram blokowy

Najprostszy typ - stała intensywność przez cały czas trwania. Używany głównie do szybkich oszacowań.

In [ ]:
# Hietogram blokowy
block = BlockHietogram()
result_block = block.generate(total_mm=total_mm, duration_min=duration_min, timestep_min=timestep_min)

print("HIETOGRAM BLOKOWY")
print(f"Liczba kroków: {result_block.n_steps}")
print(f"Intensywność na krok: {result_block.intensities_mm[0]:.2f} mm")
print(f"Intensywność średnia: {total_mm / duration_min:.4f} mm/min")
print(f"Suma kontrolna: {sum(result_block.intensities_mm):.1f} mm")

## 2. Hietogram trójkątny

Rozkład trójkątny z konfigurowalną pozycją szczytu. Parametr `peak_position` określa gdzie przypada maksimum (0.0-1.0).

In [ ]:
# Hietogram trójkątny - szczyt na początku (20%)
tri_early = TriangularHietogram(peak_position=0.2)
result_tri_early = tri_early.generate(total_mm=total_mm, duration_min=duration_min, timestep_min=timestep_min)

# Hietogram trójkątny - szczyt w środku (50%)
tri_mid = TriangularHietogram(peak_position=0.5)
result_tri_mid = tri_mid.generate(total_mm=total_mm, duration_min=duration_min, timestep_min=timestep_min)

# Hietogram trójkątny - szczyt późno (80%)
tri_late = TriangularHietogram(peak_position=0.8)
result_tri_late = tri_late.generate(total_mm=total_mm, duration_min=duration_min, timestep_min=timestep_min)

print("HIETOGRAMY TRÓJKĄTNE")
print(f"{'Pozycja szczytu':<20} {'Max intensywność':>20}")
print("-" * 42)
print(f"{'20% (wczesny)' :<20} {max(result_tri_early.intensities_mm):>20.2f} mm")
print(f"{'50% (środek)':<20} {max(result_tri_mid.intensities_mm):>20.2f} mm")
print(f"{'80% (późny)':<20} {max(result_tri_late.intensities_mm):>20.2f} mm")

## 3. Hietogram Beta

Rozkład Beta najlepiej odwzorowuje naturalne burze. Parametry:
- `alpha` - kontroluje nachylenie przed szczytem (większe = szybszy wzrost)
- `beta` - kontroluje nachylenie po szczycie (większe = wolniejszy spadek)

Typowe wartości:
- `alpha=2, beta=5` - burza z wczesnym szczytem (standardowa)
- `alpha=5, beta=2` - burza z późnym szczytem

In [ ]:
# Hietogram Beta - standardowy (wczesny szczyt)
beta_standard = BetaHietogram(alpha=2.0, beta=5.0)
result_beta_std = beta_standard.generate(total_mm=total_mm, duration_min=duration_min, timestep_min=timestep_min)

# Hietogram Beta - symetryczny
beta_sym = BetaHietogram(alpha=3.0, beta=3.0)
result_beta_sym = beta_sym.generate(total_mm=total_mm, duration_min=duration_min, timestep_min=timestep_min)

# Hietogram Beta - późny szczyt
beta_late = BetaHietogram(alpha=5.0, beta=2.0)
result_beta_late = beta_late.generate(total_mm=total_mm, duration_min=duration_min, timestep_min=timestep_min)

print("HIETOGRAMY BETA")
print(f"{'Parametry':<20} {'Max [mm]':>12} {'Czas max [min]':>15}")
print("-" * 50)
for name, result in [('alpha=2, beta=5', result_beta_std), 
                      ('alpha=3, beta=3', result_beta_sym),
                      ('alpha=5, beta=2', result_beta_late)]:
    max_idx = result.intensities_mm.argmax()
    print(f"{name:<20} {result.intensities_mm[max_idx]:>12.2f} {result.times_min[max_idx]:>15.0f}")

## 4. Porównanie wszystkich typów

In [ ]:
print("PORÓWNANIE HIETOGRAMÓW")
print("=" * 70)
print(f"{'Typ':<25} {'Sum [mm]':>10} {'Max [mm]':>12} {'Mean [mm/min]':>15}")
print("-" * 70)

hietograms = [
    ('Blokowy', result_block),
    ('Trójkątny (peak=0.4)', TriangularHietogram(0.4).generate(total_mm, duration_min, timestep_min)),
    ('Beta (2, 5)', result_beta_std),
]

for name, result in hietograms:
    total = sum(result.intensities_mm)
    max_i = max(result.intensities_mm)
    mean_i = total / duration_min
    print(f"{name:<25} {total:>10.1f} {max_i:>12.2f} {mean_i:>15.4f}")

## 5. Wizualizacja

In [ ]:
try:
    import matplotlib.pyplot as plt
    import numpy as np
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # Blokowy
    axes[0, 0].bar(result_block.times_min, result_block.intensities_mm, 
                   width=timestep_min*0.8, color='steelblue', alpha=0.7)
    axes[0, 0].set_title('Hietogram blokowy')
    axes[0, 0].set_ylabel('Intensywność [mm]')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Trójkątny
    for pos, color, label in [(0.2, 'green', 'peak=0.2'), 
                               (0.5, 'orange', 'peak=0.5'),
                               (0.8, 'red', 'peak=0.8')]:
        result = TriangularHietogram(pos).generate(total_mm, duration_min, timestep_min)
        axes[0, 1].plot(result.times_min, result.intensities_mm, 
                        'o-', color=color, label=label, alpha=0.7)
    axes[0, 1].set_title('Hietogramy trójkątne')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Beta
    for (a, b), color, label in [((2, 5), 'blue', 'α=2, β=5'),
                                   ((3, 3), 'green', 'α=3, β=3'),
                                   ((5, 2), 'red', 'α=5, β=2')]:
        result = BetaHietogram(a, b).generate(total_mm, duration_min, timestep_min)
        axes[1, 0].plot(result.times_min, result.intensities_mm, 
                        'o-', color=color, label=label, alpha=0.7)
    axes[1, 0].set_title('Hietogramy Beta')
    axes[1, 0].set_xlabel('Czas [min]')
    axes[1, 0].set_ylabel('Intensywność [mm]')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Porównanie
    axes[1, 1].bar(result_block.times_min - 1, result_block.intensities_mm, 
                   width=1.5, alpha=0.5, label='Blokowy', color='gray')
    
    tri = TriangularHietogram(0.4).generate(total_mm, duration_min, timestep_min)
    axes[1, 1].plot(tri.times_min, tri.intensities_mm, 
                    's-', label='Trójkątny', color='orange', alpha=0.8)
    
    axes[1, 1].plot(result_beta_std.times_min, result_beta_std.intensities_mm, 
                    'o-', label='Beta (2,5)', color='steelblue', linewidth=2)
    
    axes[1, 1].set_title('Porównanie typów')
    axes[1, 1].set_xlabel('Czas [min]')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Matplotlib nie jest zainstalowany. Zainstaluj: pip install matplotlib")

## Podsumowanie

| Typ | Zalety | Wady | Zastosowanie |
|-----|--------|------|-------------|
| Blokowy | Prosty | Nierealistyczny | Szybkie oszacowania |
| Trójkątny | Konfigurowalny szczyt | Liniowy rozkład | Proste modelowanie |
| Beta | Realistyczny kształt | Wymaga kalibracji | Modelowanie burz |

**Zalecenie:** Dla większości zastosowań używaj hietogramu Beta z parametrami `alpha=2, beta=5`.